## Извлекаем из html информацию

Второй файл выполняется 1 раз.
Извлекаем из html информацию и сохраняем в датафрейм.
Результат coin_data_final.csv датафрейм который содержит все данные о слабе и ссылки на изображения.

In [1]:
!pip install beautifulsoup4

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import re
import os
PATH = "D:\Python"
BASE_PATH = 'D:/Python/base.txt'

Создаем датафрейм с пустыми колонками.

In [3]:
with open(BASE_PATH, 'r') as f:
    df = pd.DataFrame(f.read().split(','),columns=['number'])
df = df.drop_duplicates()
df.head(5)

,number
0,3AF001
1,3AF002
2,3AF003
3,3AF004
4,3AF006


In [4]:
df['weight'] = ''
df['date'] = ''
df['year'] = ''
df['condition'] = ''
df['value'] = ''
df['text'] = ''
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16298 entries, 0 to 16367
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   number     16298 non-null  object
 1   weight     16298 non-null  object
 2   date       16298 non-null  object
 3   year       16298 non-null  object
 4   condition  16298 non-null  object
 5   value      16298 non-null  object
 6   text       16298 non-null  object
dtypes: object(7)
memory usage: 1018.6+ KB


Извлекаем информацию из html.

In [5]:
count = 0
bad_numbers = []
for i in tqdm(df['number']):
    try:
        with open(f"{PATH}"+f"\coin_pages_2\{i}.html", mode='r',encoding='utf-8') as file:
            soup = BeautifulSoup(file, 'html.parser')
            list = soup.find_all('div', 'col-5')
            # Возможность возобновлять процесс.
            if df.loc[df['number']==i,'value'].item() == '':
                count = count+1
                # Обработка исключений, если будет ошибка в индексах списков.
                try:
                    df.loc[df['number']==i,'date'] = list[1].text.strip()
                    df.loc[df['number']==i,'year'] = list[2].text.strip()
                    df.loc[df['number']==i,'condition'] = list[3].text.strip()
                    df.loc[df['number']==i,'weight'] = list[4].text.strip()
                    df.loc[df['number']==i,'text'] = soup.find('div',"mt-3 font-weight-bold").text
                    df.loc[df['number']==i,'value'] = soup.find('div',"col-sm-6 offset-sm-3 col-12").text
                except Exception:
                    print(f'Problem with parsing {i}')
                    bad_numbers.append(i)
                # Сохраняем прогресс каждую тысячу номеров.    
                if count%1000 == 0:
                    df.to_csv(f'{PATH}'+'\coin_data.csv')
    except Exception:
        print(f'Can not find {i}')
                
df.to_csv(f'{PATH}'+'\coin_data.csv')

 34%|█████████████████████████▉                                                   | 5493/16298 [27:53<52:31,  3.43it/s]

Problem with parsing 4NU200


 45%|██████████████████████████████████▉                                          | 7407/16298 [37:25<47:34,  3.11it/s]

Problem with parsing 4QF021


 74%|██████████████████████████████████████████████████████▋                   | 12039/16298 [1:01:50<21:00,  3.38it/s]

...

Problem with parsing 4XF946


 84%|██████████████████████████████████████████████████████████████▎           | 13732/16298 [1:10:53<14:36,  2.93it/s]

Problem with parsing 4XG893


100%|██████████████████████████████████████████████████████████████████████████| 16298/16298 [1:25:34<00:00,  3.17it/s]

Can not find 


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16298 entries, 0 to 16367
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   number     16298 non-null  object
 1   weight     16298 non-null  object
 2   date       16298 non-null  object
 3   year       16298 non-null  object
 4   condition  16298 non-null  object
 5   value      16298 non-null  object
 6   text       16298 non-null  object
dtypes: object(7)
memory usage: 1018.6+ KB


Сохраняем информацию о монетах с пустыми страницами.

In [9]:
bad_numbers = pd.DataFrame(bad_numbers, columns=['numbers'])
bad_numbers.to_csv(f'{PATH}'+'\\bad_numbers.csv')

Удаляем из основного датафрейма информацию о монетах с пустыми страницами и сохраняем.

In [29]:
df = df.loc[df['weight']!='']
df['weight'] = df['weight'].astype('float')
df['date'] = pd.to_datetime(df['date'],format="%d.%m.%Y")
df.to_csv(f'{PATH}'+'\coin_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137359 entries, 0 to 137368
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   number     137359 non-null  object        
 1   weight     137186 non-null  float64       
 2   date       137186 non-null  datetime64[ns]
 3   year       136415 non-null  object        
 4   condition  136238 non-null  object        
 5   value      137186 non-null  object        
 6   text       62505 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 8.4+ MB


Удаляем html страницы.

In [30]:
def delete_html_files(folder_path):
    # Получаем список всех файлов в папке
    files = os.listdir(folder_path)
    
    # Фильтруем только .html файлы
    html_files = [file for file in files if file.endswith('.html')]
    
    if not html_files:
        print("В папке нет .html файлов.")
        return
    
    # Запрашиваем подтверждение
    confirmation = input("Вы уверены, что хотите удалить все .html файлы? (y/n): ")
    
    if confirmation.lower() == 'y':
        for file in html_files:
            file_path = os.path.join(folder_path, file)
            os.remove(file_path)
    else:
        print("Удаление отменено.")

# Укажите путь к папке
folder_path = f"{PATH}"+"\coin_pages_2"
delete_html_files(folder_path)

Вы уверены, что хотите удалить все .html файлы? (y/n):  y
